## Scoping

# ETL (Extract, Transform, Load)

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sb
import folium
plt.rcParams['figure.figsize'] = [12, 8]

## Yield

In [ ]:
df_sk=pd.read_csv('/Users/ruhidmirzayev/Palette/Data/rm-yields-data.csv')

In [ ]:
df_mb=pd.read_excel('/Users/ruhidmirzayev/Palette/Data/MMPP - Yield by Soil Type Browser.xlsx')

## Shapefiles

In [ ]:
gdf_sk=gpd.read_file('/Users/ruhidmirzayev/Palette/Data/Shapefiles/SK_RM_Shapefiles/RuralMunicipality.shp')

In [ ]:
gdf_mb=gpd.read_file('/Users/ruhidmirzayev/Palette/Data/Shapefiles/MB_RM_Shapefiles/MB_Municipal_Boundaries.shp')

## Transforming

In [ ]:
# List of columns
df_sk.columns

In [ ]:
# List of columns
df_mb.columns

In [ ]:
# Info about columns
df_sk.info()

In [ ]:
# Info about columns
df_mb.info()

In [ ]:
# Table heads
df_sk.head()

In [ ]:
# Table heads
df_mb.head()

In [ ]:
df_mb['Yield/acre(Metric)']=df_mb['Yield/acre(Metric)'].str.replace(' Tonnes', '') # Replacing Tonnes
df_mb['Yield/acre(Metric)']=df_mb['Yield/acre(Metric)'].replace('Tolerance', np.NaN) # Replacing Tolerance
df_mb['Yield/acre(Metric)']=df_mb['Yield/acre(Metric)'].astype(float) # changing object to float data type

In [ ]:
df_mb_pivot=pd.pivot_table(df_mb.drop(columns=['Yield/acre(Metric).1', 'Yield/acre(Imperial)', 'Soil', 'Farms' ]),
               index=['Risk Area / R.M.', 'Year'], columns='Crop', values='Yield/acre(Metric)')\
               .reset_index()

In [ ]:
df_sk.columns

In [ ]:
df_mb_pivot.columns

In [ ]:
df_mb_clean=df_mb_pivot.rename(columns={
    'Risk Area / R.M.': 'RM', 
    'ARGENTINE CANOLA': 'Canola', 
    'BARLEY':'Barley',
    'CANARYSEED': 'Canary Seed', 
    'DURUM WHEAT': 'Durum Wheat', 
     'LENTILS': 'Lentils', 
     'OATS': 'Oats',
    'RED SPRING WHEAT': 'Spring Wheat', 
    'WHITE PEA BEANS': 'Peas'}) \
        .drop(columns=['ALFALFA', 'FABABEANS', 'FABABEANS', 'POLISH CANOLA'] )

In [ ]:
df_sk_clean=df_sk.rename(columns={'Durum': 'Durum Wheat'}).drop(columns=['Winter Wheat', 'Mustard', 'Sunflowers', 'Fall Rye', 'Spring Rye', 'Tame Hay','Flax', 'Chickpeas' ] )

In [ ]:
df_sk_clean

In [ ]:
df_mb_clean

In [ ]:
# Crop conversion in MB tonnes to bushel
df_mb_clean['Canola']=df_mb_clean['Canola'] * 44.092
df_mb_clean['Barley']=df_mb_clean['Barley'] * 45.93
df_mb_clean['Canary Seed']=df_mb_clean['Canary Seed'] * 44.092
df_mb_clean['Durum Wheat']=df_mb_clean['Durum Wheat'] * 36.74
df_mb_clean['Lentils']=df_mb_clean['Lentils'] * 36.74
df_mb_clean['Oats']=df_mb_clean['Oats'] * 64.842
df_mb_clean['Spring Wheat']=df_mb_clean['Spring Wheat'] * 36.74
df_mb_clean['Peas']=df_mb_clean['Peas'] *  36.74


In [ ]:
# Crop Conversion in SK - pounds to bushels
df_sk_clean['Lentils']=df_sk_clean['Lentils'] / 60
df_sk_clean['Canary Seed']=df_sk_clean['Canary Seed'] / 50

In [ ]:
# Creating province column
df_mb_clean['Province'] = 'MB'
df_sk_clean['Province'] = 'SK'

In [ ]:
df = pd.concat([df_mb_clean, df_sk_clean])

# EDA(Exploratory Data Analysis)

## Missing Values

In [ ]:
df.isna().sum().sort_values().plot(kind='bar', color='green')
plt.title('Missing Values- 1938 to 2022')
plt.xlabel('Crops')
plt.ylabel('# of Missing values')
plt.axhline(len(df)/2, linestyle='--', color='red')
plt.axhline(len(df)/4, linestyle='--', color='red')
plt.axhline(len(df)/10, linestyle='--', color='red')
plt.show()

In [ ]:
df.loc[df['Year']>2002].isna().sum().sort_values().plot(kind='bar', color='green')
plt.title('Missing Values - 2002 to 2022')
plt.xlabel('Crops')
plt.ylabel('# of Missing values')
plt.axhline(len(df)/2, linestyle='--', color='red')
plt.axhline(len(df)/4, linestyle='--', color='red')
plt.axhline(len(df)/10, linestyle='--', color='red')
plt.show()

## Outliers

In [ ]:
crop_columns=['Canola', 'Barley', 'Canary Seed', 'Durum Wheat',
       'Lentils', 'Oats', 'Spring Wheat', 'Peas']

## Histograms

In [ ]:
df[crop_columns].hist(bins=300)
plt.show()

In [ ]:
sb.heatmap(df[crop_columns].corr(), annot=True)

## Boxplots

In [ ]:
df[crop_columns].boxplot()

In [ ]:
df.loc[df['Spring Wheat']>180]

In [ ]:
# Peorson Corr (-1 to 1), -1 negative corr, o no corr, 1 positive corr
# Using Seaborn
sb.heatmap(df[crop_columns].corr(),annot=True, cmap='Greens')


## GIS Analyis

In [ ]:
# CRS --> Cordinate Reference Systems
gdf_mb['geometry'].crs

In [ ]:
# Standardazing CRS formats
gdf_sk['geometry']=gdf_sk['geometry'].to_crs('epsg:4326')
gdf_mb['geometry']=gdf_mb['geometry'].to_crs('epsg:4326')

In [ ]:
gdf_sk['RMNO']=gdf_sk['RMNO'].astype(int)

In [ ]:
#set(gdf['RM'].unique()) - set(df_sk_clean['RM'].unique())

In [ ]:
gdf_mb['MUNI_NAME']=gdf_mb['MUNI_NAME'].str.replace('RM OF ','')

In [ ]:
gdf=pd.concat([gdf_sk[['RMNO','geometry']].rename(columns={'RMNO':'RM'}),gdf_mb[['MUNI_NAME', 'geometry']].rename(columns={'MUNI_NAME':'RM'})])

In [ ]:
# gdf.plot()not interactive
# gdf.explore()

In [ ]:
gdf['RM']=gdf['RM'].astype('string')
df['RM']=df['RM'].astype('string')

In [ ]:
df_gdf=pd.merge(gdf, df, on='RM')

In [ ]:
print('Before merging', gdf['RM'].nunique())
print('After merging',df_gdf['RM'].nunique())

In [ ]:
ds=df_gdf.copy()

### GIS Visualization

#### Explore functiom

In [ ]:
m=ds.loc[ds['Year']==2021].explore(column='Canola', 
                                 legend=True,  
                                 cmap='Greens',
                                 tooltip= ['Canola', 'RM'],
                                 tiles='Stamen Toner') # Plot() is good for showing up in GitHub, Explore() is good for interactive map and saving as HTML

# Adding a title with dark orange color to the folium map
title_html = '''
                 <h3 align="center" style="font-size:30px; color:Green;"><b> Canola Yield in 2021 </b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))
m.save('/users/ruhidmirzayev/Palette/Canola_2021.html')

#### Plot

In [ ]:
ds.loc[ds['Year']==2021].plot(column='Canola', 
                                 legend=True,  
                                 cmap='Greens')

## Tableau

## Aggragetions

In [ ]:
df_03_22=df.loc[df['Year']>2002].sort_values(['RM', 'Year'])

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(5, 4, figsize=(15, 20))
years = df_03_22['Year'].unique()

for i, year in enumerate(years):
    ax = axes[i//4, i%4]
    merged_df = pd.merge(gdf, df_03_22.loc[df_03_22['Year'] == year], on='RM')
    merged_df.plot(column='Spring Wheat', cmap='RdYlGn', legend=True, ax=ax)
    ax.set_title(f'Spring Wheat Yield in {year}', color='Blue', size=12)

plt.tight_layout()
plt.show()

In [ ]:
df_03_22.columns

In [ ]:
crop_columns

In [ ]:
agg_funcs = {
    column: ['mean', 'std', 'median'] for column in crop_columns
}

df_agg = df_03_22.set_index('RM')[crop_columns].groupby('RM').agg(agg_funcs)

# Optionally, to flatten the multi-level columns:
df_agg.columns = ['_'.join(col).strip() for col in df_agg.columns.values]

# Dropping missing values for Canola mean.
df_agg=df_agg.dropna(subset='Canola_mean')

# Feature Selection

## Filtered Methods

In [ ]:
# ANOVA
# Pearson Correlation

## Wrapper

In [ ]:
## Recurisive Feature Elimnation
## Backward feature elimination

## Emedded 

In [ ]:
# Decision Tree
# Lasso Reg

# Standardizing/Split

In [ ]:
# Only for Supervised ML, not unsupervised

# Training Models

In [ ]:
# Use default parameters - not advised

# Parameter tuning
# Use always grid search

## K-Means Clustering

In [ ]:
# Importing library
from sklearn.cluster import KMeans 

df_agg_can= df_agg[['Canola_mean', 'Canola_std']]

# Let's define our features
X = df_agg_can.copy()

from sklearn.metrics import silhouette_score
n_clusters = [2,3,4,5,6,7,8,9,10,11,12,13,14,15] # number of clusters
clusters_inertia = [] # inertia of clusters
s_scores = [] # silhouette scores

for n in n_clusters:
    KM = KMeans(n_clusters=n, init='k-means++').fit(X)
    clusters_inertia.append(KM.inertia_)    # data for the elbow method
    silhouette_avg = silhouette_score(X, KM.labels_)
    s_scores.append(silhouette_avg) # data for the silhouette score method

## Elbow Metod

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
ax.plot(n_clusters, clusters_inertia, 'o-', color='blue', label='Elbow Method')
ax.set_title("Elbow Method")
ax.set_xlabel("Number of Clusters")
ax.set_ylabel("Clusters Inertia")
ax.axvline(4, ls="--", c="red")
ax.axvline(5, ls="--", c="red")
ax.axvline(6, ls="--", c="red")
plt.grid()
plt.legend()
plt.show()

## Silhouette Score

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
ax.plot(n_clusters, s_scores, 's-', color='green', label='Silhouette Score Method')
ax.set_title("Silhouette Score Method")
ax.set_xlabel("Number of Clusters")
ax.set_ylabel("Silhouette Score")
ax.axvline(6, ls="--", c="red")
plt.grid()
plt.legend()
plt.show()


In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, init='k-means++', random_state=42)
df_agg_can['Clusters_5']=kmeans.fit_predict(df_agg_can)

In [ ]:
kmeans = KMeans(n_clusters=7, init='k-means++', random_state=42)
df_agg_can['Clusters_7']=kmeans.fit_predict(df_agg_can)

In [ ]:
sb.scatterplot(data=df_agg_can, x='Canola_mean', y='Canola_std', hue='Clusters_5')
plt.title('Canola Clustering Mean and Std | 2003-2022 | K-Means ', color='blue', size =14)
plt.show()

In [ ]:
sb.scatterplot(data=df_agg_can, x='Canola_mean', y='Canola_std', hue='Clusters_7')
plt.title('Canola Clustering Mean and Std | 2003-2022 | K-Means ', color='blue', size =14)
plt.show()

In [ ]:
pd.merge(
    gdf,
    df_agg_can,
    on='RM'
).explore(column='Clusters_5', legend='True', k=5, scheme='naturalbreaks', cmap='Oranges')

## Ranking clusters based on Mean

In [ ]:
df_agg_can.groupby('Clusters_5').mean()\
    .sort_values('Canola_mean')

In [ ]:
# Ranking based on the mean
df_agg_can['Clusters_5_ranked']=df_agg_can['Clusters_5'].replace(to_replace={
    4:0,
    0:1,
    2:2,
    1:3,
    3:4
})

In [ ]:

pd.merge(
    gdf,
    df_agg_can,
    on='RM'
).explore(column='Clusters_5_ranked', legend='True', k=5, scheme='naturalbreaks', cmap='Oranges')

In [ ]:
# Compute mean for each cluster
cluster_means = df_agg_can.groupby('Clusters_7').mean()['Canola_mean']

# Get the rank for each cluster based on the mean
cluster_ranks = cluster_means.rank().astype(int) - 1  # Subtract 1 to make ranking start from 0

# Map the original cluster labels to their corresponding ranks
df_agg_can['Clusters_7_ranked'] = df_agg_can['Clusters_7'].map(cluster_ranks)


In [ ]:
pd.merge(
    gdf,
    df_agg_can,
    on='RM'
).explore(column='Clusters_7', legend='True', k=7, scheme='naturalbreaks', cmap='Oranges')

In [ ]:
pd.merge(
    gdf,
    df_agg_can,
    on='RM'
).explore(column='Clusters_7_ranked', legend='True', k=7, scheme='naturalbreaks', cmap='Oranges')

# Error  (Supervised ML)

In [ ]:
# MAE (Mean Absolute Error)
# RMSE(Mean Squared Error)

# Based on above errors, find the min error model.
# Check error distribution
# Look at the difference
# Make a scatterplot

# Deployment

In [ ]:
# ML engineers deploys models with SD

# AWS Sagemaker

# Monitoring

In [ ]:
# Look at error

# If something wrong go back to step 1

# Joining Tables based on  location in Geopandas

In [ ]:
import geodatasets
chicago = gpd.read_file(
    geodatasets.get_path("geoda.chicago_health")
)
groceries = gpd.read_file(
    geodatasets.get_path("geoda.groceries")
).to_crs(chicago.crs)

In [ ]:
chicago.plot()

In [ ]:
chicago.plot()
groceries.plot()


In [ ]:
import matplotlib.pyplot as plt

# Assuming chicago and groceries are already loaded GeoDataFrames

fig, ax = plt.subplots(figsize=(10, 10))

# Plot the chicago GeoDataFrame
chicago.plot(ax=ax, color='blue', edgecolor='black')

# Overlay the groceries GeoDataFrame
groceries.plot(ax=ax, color='red', markersize=10, label='Grocery Stores')

# Add title and legend (if desired)
ax.set_title("Chicago with Grocery Stores")
ax.legend()

plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming chicago and groceries are already loaded GeoDataFrames

fig, ax = plt.subplots(figsize=(10, 10))

# Plot the chicago GeoDataFrame
chicago.plot(ax=ax, color='blue', edgecolor='black')

# Overlay the groceries GeoDataFrame
groceries.plot(ax=ax, color='red', edgecolor='white', label='Groceries')

# Add title and legend
ax.set_title("Chicago with Groceries")
ax.legend()

plt.show()



In [ ]:
gpd.sjoin(groceries, chicago,  predicate='within').head(5)

In [ ]:
groceries.head()